In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
import os
import ast
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import random
import lime
random.seed(1993)
random.seed(1993)
%matplotlib inline

In [2]:
df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
df_test

,text,spans
0,Evanukachum pondati kolandhainu sentiment irun...,"[71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 8..."
1,Adei ennada short film la irunthu suturukinga,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]"
2,Super dialogue Oruthar mela visvasam kattrath...,"[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7..."
3,Epadiya jathi padam. Ponnu padama edungada. In...,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
4,Ponnu mella kaivaikuravan Kai mattum illa uyir...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
...,...,...
871,காலா.காபலி.அசுரன்.பாரியேரும்.பெருமள் வந்த அப்ப...,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
872,Kekka bekka short film mathri irukey,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
873,Bayangaram... Trailerey ippadina appa Padam en...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
874,komali rasini vesam pottu tamil ilichavayangal...,"[0, 1, 2, 3, 4, 5, 6, 7, 31, 32, 33, 34, 35, 3..."


In [3]:
# Complex + Multilabel

In [4]:
# Read the output and create spans

In [5]:
def f1_span(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        if len(predictions) == 0:
            return 0
        else:
            return 1
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [6]:
# Complex-Multi

In [70]:
#BERT ONLY

import re

def eval_multiple(LIMEFILE,MAXSIZE,MINSIZE):
    
    output_xlm_cm = np.load(LIMEFILE,allow_pickle=True)
    new_output_xlm=[]
    df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
    
    start=0
    full_res=[]
    for index in range(len(output_xlm_cm)):
        modified=[]
        temp = output_xlm_cm[index]["LABEL_0"][1:-1]
        start=0
        for w,s in temp:
            if "##" in w:
                modified.append((w,s,[i for i in range(start,start+len(w)-2)]))
                start+=len(w)-2
            elif "_" in w:
                modified.append((w,s,[i for i in range(start,start+len(w)-1)]))
                start+=len(w)-1
            else:
                modified.append((w,s,[i for i in range(start,start+len(w))]))
                start+=len(w)
        full_res.append(modified)
    
    THRESH=-200
    
    totalscore=[]
    for index in range(len(full_res)):
        if len(df_test["text"].values.tolist()[index])>=MINSIZE and len(df_test["text"].values.tolist()[index])<=MAXSIZE:
            gt = df_test["spans"].values.tolist()[index]
            predspan=[]
            for word,score,sp in full_res[index]:
                if score>=THRESH:
                    predspan+=sp
            sc = f1_span(predspan, ast.literal_eval(gt))
            totalscore.append(sc)
    
    print("Results:",np.average(totalscore))
    
LIMEFILE = "../models/complex_cue_multi/offensive_xlm_complex_aug_multi/xlm_roberta_attribution_complex_multi_all.npy"

MAXSIZE = 10000
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 30
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 50
MINSIZE = 30
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 500
MINSIZE = 50
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

Results: 0.42067713023396935
Results: 0.4926620524160055
Results: 0.47332544655541625
Results: 0.3974326328958221


In [ ]:
# Baseline and Complex

In [11]:
#BERT ONLY

import re

def eval_multiple(LIMEFILE,MAXSIZE,MINSIZE):
    
    output_xlm_cm = np.load(LIMEFILE,allow_pickle=True)
    new_output_xlm=[]
    df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
    
    start=0
    full_res=[]
    for index in range(len(output_xlm_cm)):
        modified=[]
        
        temp = output_xlm_cm[index][1:-1]
        start=0
        for w,s in temp:
            if "##" in w:
                modified.append((w,s,[i for i in range(start,start+len(w)-2)]))
                start+=len(w)-2
            elif "_" in w:
                modified.append((w,s,[i for i in range(start,start+len(w)-1)]))
                start+=len(w)-1
            else:
                modified.append((w,s,[i for i in range(start,start+len(w))]))
                start+=len(w)
        full_res.append(modified)
    
    THRESH=-100
    
    totalscore=[]
    for index in range(len(full_res)):
        if len(df_test["text"].values.tolist()[index])>=MINSIZE and len(df_test["text"].values.tolist()[index])<=MAXSIZE:
            gt = df_test["spans"].values.tolist()[index]
            predspan=[]
            for word,score,sp in full_res[index]:
                if score>=THRESH:
                    predspan+=sp
            sc = f1_span(predspan, ast.literal_eval(gt))

            
            totalscore.append(sc)
    
    print("Results:",np.average(totalscore))
    
LIMEFILE = "../models/baseline/offensive_xlm_baseline/xlm_roberta_attribution_baseline.npy"

# MAXSIZE = 10000
# MINSIZE = 0
# eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 1000
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

# MAXSIZE = 50
# MINSIZE = 30
# eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

# MAXSIZE = 500
# MINSIZE = 50
# eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

NameError: name 'lime_pred' is not defined